In [3]:
import sys
import csv
print(sys.version, sys.platform, sys.executable)
from tqdm import tqdm

import pandas as pd

3.6.10 (default, Dec 19 2019, 23:04:32) 
[GCC 5.4.0 20160609] linux /usr/bin/python3


In [5]:
import sys
import csv
print(sys.version, sys.platform, sys.executable)
from tqdm import tqdm

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.precision',16)

train_path = './'
test_path = './'


col_name = ['item_id']
for i in range(0, 128):
    col_name.append('tv' + str(i))
for i in range(0, 128):
    col_name.append('iv' + str(i))

itemft = pd.read_csv(train_path + 'underexpose_item_feat.csv', low_memory=False, names=col_name)
itemft = itemft.replace('[\[\]]', '', regex=True)  # regex=True is the key

vec = itemft.iloc[:, 1:].to_numpy()
vec.shape



chunk_size = 500
matrix_len = vec.shape[0]


def sim_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(vec[start:end], vec)


import numpy as np

sim_matrix = np.ndarray(shape=(matrix_len, matrix_len))

# ---------------------------------------------------------------------

for start in tqdm(range(0, matrix_len, chunk_size)):
    cos_sim_chunk = sim_chunk(start, start + chunk_size)
    if (start + chunk_size) <= matrix_len:
        sim_matrix[start:start + chunk_size] = cos_sim_chunk
    else:
        sim_matrix[start:matrix_len] = cos_sim_chunk



3.6.10 (default, Dec 19 2019, 23:04:32) 
[GCC 5.4.0 20160609] linux /usr/bin/python3


100%|██████████| 218/218 [04:08<00:00,  1.14s/it]


In [6]:

results = {}
recall_num=500
for idx, row in tqdm(itemft[['item_id']].iterrows(),total=sim_matrix.shape[0]):
    similar_indices = sim_matrix[idx].argsort()[
                      :-recall_num:-1]  # ascending return index, : means reverse order, -100:-1 means last 100-1 to last 1-1 position
    similar_items = [(sim_matrix[idx][i], itemft['item_id'][i]) for i in similar_indices]

    results[row['item_id']] = similar_items[1:]
    
# with open('result_vec.txt','w') as f:
#     print(results,file=f)

# with open(file_path) as csv_file:
#     reader = csv.reader(csv_file)
#     mydict = dict(reader)

100%|██████████| 108916/108916 [30:18<00:00, 59.88it/s]


In [7]:

file_path='all_vec_result-5-31.csv'
with open(file_path, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in results.items():
        writer.writerow([key, value])
        

In [14]:
all_click=pd.DataFrame()
now_phase=6
for p in tqdm(range(0,now_phase+1)):
    user_last = pd.read_csv(test_path + 'underexpose_test_click-{}.csv'.format(p),names=['user_id','item_id','time'])
    print(user_last.shape)
    all_click=all_click.append(user_last)
    print(all_click.shape)
all_click.drop_duplicates(inplace=True)    
user_last=all_click.sort_values(['user_id', 'time'],ascending=[True,False]).reset_index(drop=True)
user_train = user_last.drop_duplicates(['user_id'], keep='first')

100%|██████████| 7/7 [00:00<00:00, 55.78it/s]


(21216, 3)
(21216, 3)
(24465, 3)
(45681, 3)
(22745, 3)
(68426, 3)
(24021, 3)
(92447, 3)
(26386, 3)
(118833, 3)
(27653, 3)
(146486, 3)
(28788, 3)
(175274, 3)


In [15]:
user_train.shape

(12081, 3)

In [16]:
recall_num=450
itemlist=list(itemft.item_id.unique())
recall_frame = pd.DataFrame(columns=['user_id', 'item_id_pred','rank'],index=range(0,recall_num*len(user_train.user_id.unique())))
row_num=0
for user in tqdm(user_train.user_id.unique()):
    latest_item = user_train.loc[user_train.user_id == user, 'item_id'].values[0]
    if latest_item in itemlist: 
        recs = results[latest_item][:recall_num]
        for i in range(0, recall_num):
            recall_frame.iloc[row_num]['user_id'] = user
            recall_frame.iloc[row_num]['item_id_pred'] = recs[i][1]
            recall_frame.iloc[row_num]['rank'] = i+1
            row_num+=1
    else:
        recall_frame.iloc[row_num]['user_id'] = user
        recall_frame.iloc[row_num]['item_id_pred'] = np.nan
        recall_frame.iloc[row_num]['rank'] = np.nan
        row_num+=1
recall_frame
recall_frame=recall_frame.dropna(how='all').astype('float32')

recall_frame.to_csv('recall_allvec_df.csv',index=False)

100%|██████████| 12081/12081 [26:50<00:00,  7.50it/s]


In [18]:
recall_frame.shape

(4822667, 3)

In [22]:
pred_df=pd.read_csv('recall_allvec_df.csv',low_memory=False, quoting=csv.QUOTE_NONE, error_bad_lines=False)
pred_df=pred_df.dropna(subset=['item_id_pred'])
pred_df=pred_df.loc[pred_df['rank'] <=50 ]
pred_df.shape

(535700, 3)

In [23]:
12081*50 - 535700

68350

In [37]:
top50_items[::-1]

[102105,
 110172,
 90006,
 78831,
 69717,
 14239,
 4483,
 899,
 80227,
 51182,
 8149,
 42884,
 29594,
 35264,
 114125,
 67536,
 13845,
 41755,
 43439,
 35452,
 2611,
 49787,
 29931,
 73761,
 113569,
 24025,
 78364,
 14777,
 59903,
 91838,
 4999,
 1518,
 44534,
 87254,
 12061,
 6568,
 77047,
 34716,
 42651,
 1604,
 4230,
 7156,
 2420,
 21279,
 21426,
 111225,
 13302,
 4340,
 12845,
 52766]

In [38]:
preds={}
for user in tqdm(pred_df.user_id.unique()):
    assert len(pred_df.loc[pred_df.user_id == user ]) == 50
    index_list = pred_df.loc[pred_df.user_id == user].index[:50]
    preds[int(user)] = pred_df.loc[index_list].item_id_pred.astype(int).to_list()

top50_items = [52766, 12845, 4340, 13302, 111225, 21426, 21279, 2420, 7156, 4230, 1604, 42651, 34716, 77047, 6568, 12061, 87254, 44534, 1518, 4999, 91838, 59903, 14777, 78364, 24025, 113569, 73761, 29931, 49787, 2611, 35452, 43439, 41755, 13845, 67536, 114125, 35264, 29594, 42884, 8149, 51182, 80227, 899, 4483, 14239, 69717, 78831, 90006, 110172, 102105]
user_to_be_filled= set(user_train.user_id) - preds.keys()
for user in tqdm(user_to_be_filled):
    preds[user] = top50_items[::-1]

100%|██████████| 1367/1367 [00:00<00:00, 440606.59it/s]


In [39]:
submit=pd.DataFrame.from_dict(data=preds,orient='index')
submit.reset_index()
# submit.replace('[\[\]]',regex=True)
submit.to_csv('submit.csv',header=False)

In [ ]:
score:0.1481
hitrate_50_full:0.3855
ndcg_50_full:0.1481
hitrate_50_half:0.3519
ndcg_50_half:0.1398